In [49]:
%pylab inline
import numpy as np
import scipy as sc
import pandas as pd
from ABC_algorithm import ABC

import statsmodels as sm
from sklearn.linear_model import LinearRegression

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['mod']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [61]:
######
# set up for the normal ABC example
######

prior_mean = -4.0
prior_sd = 3
likelihood_sd = 1

def NormalPriorSampler(n):
    return np.random.normal(loc=prior_mean, scale=prior_sd, size=n)

def NormalLiklihoodSimulator(n, param):
    #unknown mean
    return np.random.normal(loc=param, scale=likelihood_sd, size=n)
    
def NormalSummary(data):
    return np.mean(data, axis=0)

data = np.random.normal(loc=0,scale=likelihood_sd,size=5)

post_sd = np.power((1.0/np.power(prior_sd,2) + len(data)/np.power(likelihood_sd,2)),-1/2)
post_mean = (prior_mean/np.power(prior_sd,2) + np.sum(data)/(np.power(likelihood_sd,2)))/(1/np.power(prior_sd,2) + len(data)/np.power(likelihood_sd,2))
print post_mean, "is posterior mean"
print post_sd, "is posterior std dev"

df = ABC(NormalPriorSampler, NormalLiklihoodSimulator, NormalSummary, 0.5, data, 1000)

-1.00796685062 is posterior mean
0.195652173913 is posterior std dev


In [62]:
def EpanechnikovKernel(t,delta,c=1):
    if t<=delta:
        return c*(1/delta)*(1-(t/delta)**2)
    else:
        return 0

## A function for post processing ABC algorithm output

In [145]:
def post_process(abc_df, q=0.5):
    """
    Given a dataframe output to ABC, return dataframe with accepted thetas transformed in column "theta_star"
    """
    # take accpted values output from ABC
    df_accepted = abc_df[abc_df['accept'] == 1]

    df_accepted.statistics_diff = df_accepted.statistics - NormalSummary(data)
    df_accepted.statistics_diff_abs = df_accepted.statistics_diff.apply(lambda x: abs(x))
    
    #df_accepted.statistics_diff_abs.hist(bins=100)
    quantile = df_accepted.statistics_diff_abs.quantile(q)
    #plt.show()

    #create column with kernal transform
    df_accepted.kernel = df_accepted.statistics_diff.apply(lambda x: EpanechnikovKernel(abs(x), delta=quantile))

    #print df_accepted.kernel[df_accepted.kernel > 0.0]

    mod = LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
    X = np.array(df_accepted.statistics_diff)
    X.shape = (shape(X)[0],1)
    y = np.array(df_accepted.theta)
    y.shape = (shape(y)[0],1)
    weights = np.array(df_accepted.kernel)
    
    res = mod.fit(X, y, sample_weight=weights)
    #alpha = res.intercept_[0]
    beta = res.coef_[0][0]
    print beta
    beta_stats_diff = beta*df_accepted.statistics_diff
    beta_stats_diff_array = np.array(beta_stats_diff)
    beta_stats_diff_array.shape = (shape(beta_stats_diff_array)[0],1)
    theta_star = np.subtract(y, beta_stats_diff_array)
    theta_star = np.reshape(theta_star,83)
    print np.shape(theta_star)
    df_accepted['theta_star'] = theta_star
    
    return df_accepted

In [147]:
df_post_processed = post_process(df, 0.5)
print df_post_processed.theta_star

1.19020438302
(83,)
2     -1.170073
17    -1.511627
18    -1.380540
27    -0.600571
29    -0.902872
45    -0.517177
49    -0.647777
53    -0.426499
56    -0.986676
71    -0.607288
141   -0.445191
145   -1.225733
161   -0.485804
176   -0.629761
189   -0.989381
209   -1.229147
226   -1.736271
264    0.157817
282   -1.466223
317   -0.827487
322   -1.206964
326   -1.349115
338   -0.706378
352   -1.620570
359   -1.511238
362   -2.203744
373   -1.204515
395   -0.109563
404   -1.483605
415   -0.814308
         ...   
695   -1.253886
696   -0.005825
707   -0.755784
723   -1.207014
755   -0.828844
757   -1.072362
758   -1.408224
763   -1.328564
779   -1.537918
789   -1.086265
803   -0.305261
805   -0.672450
821   -0.351747
832   -0.545506
843   -1.023778
848   -1.305956
862   -1.036729
866   -1.072856
868   -0.524924
901   -1.045816
912   -0.860620
918   -1.572213
920   -1.202170
946   -0.444738
957   -0.497915
967   -2.043097
970   -1.396672
984   -0.470495
988   -1.581361
999   -0.845208
Name

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
